# SERVIDOR TCP2

APARTADO 2.1

- En este apartado el cliente va a solicitar un fichero al archivo para ver su contenido, mediante el comando "get NOMBRE_FICHERO"
- Para terminar la conexión el cliente enviará un "Bye"
- Si el fichero solicitado no existe 

In [3]:
import socket 
import sys
import time

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_address = ("localhost", 6780)
s.bind(server_address)
s.listen(1)
conn, address = s.accept()
msg = ''
while str(msg)!='Bye':
    try:
        data = conn.recv(1024).decode()
        msg = data
        print('RX: ', data)
        if msg != 'Bye':
            #conn.sendall('HOLA CLIENTE'.encode())
            print('El fichero que se desea abrir es: '+ data)
            fichero = open(str(data), 'rb')
            f = fichero.read(1024)
            print(f)
            conn.sendall(f)
    except:
        print('El fichero no existe')
        conn.sendall('El fichero no existe'.encode())
    
print('Conexión cerrada')
conn.close()

RX:  asid
El fichero que se desea abrir es: asid
El fichero no existe
RX:  Bye
Conexión cerrrada


####################################################################################################

APARTADO 2.2

- En este apartado se amplia el código anterior para que el cliente pueda hacer dos tipos de peticiones.
- Petición I - get:
    Al igual que en el apartado anterior el cliente puede pedirle al servidor que le envía un archivo par aver su contenido
    Si el archivo no existe, el servidor crea el fichero
- Petición II - put:
    Si el cliente manda el comando put, el servidor sigue los siguientes pasos:
        1) mirar si existe el fichero, si NO exite lo crea
        2) bloquear fichero
        3) enviar fichero en formato rb
        4) recibir cambios del cliente
        5) aplicar cambios abriendo el archivo otra vez --> ab
        6) desbloquear fichero
- En el caso de que el cliente quiera terminar la conexión envía el comando "Bye"
- Ya hay un archivo creado, llamado prueba.txt que tiene algo de contenido. En el diccionario donde están los archivos
  bloquados se ha insertado este archivo con un puerto aleatorio para ver si el bloqueo del fichero funcionaba correctamente.
  Si se desea probar los otros comandos mejor crear un archivo nuevo.

In [2]:
import socket 
import sys
import time    
    
def funcion_get(nombre):
    fichero = open(instruccion[1], 'rb')
    contenido = fichero.read(1024)
    fichero.close()
    
    return contenido
    
    
def funcion_update(nombre, modificacion):
    fichero = open(nombre, 'a')
    fichero.write(modificacion.decode())
    fichero.close()
    
def funcion_comprobacion_existencia(nombre):
    res = True
    try:
        contenido = funcion_get(nombre)
    except:
        f = open(nombre, 'x')
        f.close()
        res = False
    
    return res
        
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_address = ("localhost", 6780)
s.bind(server_address)
s.listen(1)

diccionario = {'prueba.txt':('127.0.0.1', 2346)} # guardamos los archivo que están siendo modificados
while True:
    conn, address = s.accept()
    data = conn.recv(1024).decode()
    
    if data == 'Bye':
        break
    
    instruccion = data.split() # es un vector de strings
    creado = funcion_comprobacion_existencia(instruccion[1])
    
    if creado:
        if instruccion[0] == 'get':
            # servidor devuelve el contenido del fichero
            contenido = funcion_get(instruccion[1])
            print('Enviando fichero al cliente...')
            conn.sendall(contenido)
            
        elif instruccion[0] == 'put':
            # 1) bloquear fichero
            # 2) enviar fichero en formato rb
            # 3) recibir cambios del cliente
            # 4) aplicar cambios abriendo el archivo otra vez --> ab
            # 5) desbloquear fichero
            conn.sendall('El archivo está listo para ser modificado.'.encode())
            
            if instruccion[1] in diccionario:
                # el archivo está en uso
                if diccionario.get(instruccion[1]) == address:
                    # el usuario puede modificarlo    
                    conn.sendall('Introduzca la modificación: '.encode())
                    
                    texto_modificacion = conn.recv(1024)
                    funcion_update(instruccion[1], texto_modificacion)

                    # desbloquear el archivo --> poner a 0
                    diccionario.pop(instruccion[1], address)

                    contenido = funcion_get(instruccion[1])
                    print('Enviando fichero al cliente...')
                    conn.sendall(contenido)
                    print('Modificación efectuada con éxito.')
                else:
                    # el usuario NO puede modificarlo
                    conn.sendall('El archivo esta siendo modificado, espere su turno.'.encode())
            else:
                # primera vez que se hace put en archivo
                # bloqueamos el archivo --> poner a 1
                conn.sendall('Introduzca la modificación: '.encode())
                
                diccionario.update({instruccion[1]:address})
    
                texto_modificacion = conn.recv(1024)
                funcion_update(instruccion[1], texto_modificacion)

                # desbloquear el archivo --> poner a 0
                diccionario.pop(instruccion[1], address)

                contenido = funcion_get(instruccion[1])
                print('Enviando fichero al cliente...')
                conn.sendall(contenido)
                print('Modificación efectuada con éxito.')
    else:
        conn.sendall('El archivo no estaba creado.'.encode())
    
print('Conexión cerrada')
conn.close()

Enviando fichero al cliente...
Enviando fichero al cliente...
Modificación efectuada con éxito.
Enviando fichero al cliente...


IndexError: list index out of range